In [ ]:

import os, shutil, sys

import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import array_to_img

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
from PIL import Image
import cv2
import shutil

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


In [3]:
#get working directory
os.getcwd()

'C:\\Users\\Mojtaba Jafari Tadi\\Projects_AMK\\Radiology Project\\fatpad'

In [4]:
os.listdir()

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 'brucechou1983_CheXNet_Keras_0.3.0_weights.h5',
 'CNN_automated_medical_diagnostics_five_layers_chopped_two_classes_trained_on_twenty_epochs_Adam_0.0001.ipynb',
 'CNN_FATPAD_MultiHeadAttention.ipynb',
 'CNN_FATPAD_ResidualAttention.ipynb',
 'CNN_FATPAD_SimplePlainNN.ipynb',
 'CNN_FATPAD_TransferLearning-DenseNet.ipynb',
 'CNN_FATPAD_TransferLearning-EfficientNet.ipynb',
 'CNN_FATPAD_TransferLearning-Inception.ipynb',
 'CNN_FATPAD_TransferLearning-MobileNetSE.ipynb',
 'CNN_FATPAD_TransferLearning-NASNet.ipynb',
 'CNN_FATPAD_TransferLearning-ResNet.ipynb',
 'CNN_FATPAD_TransferLearning-Xception.ipynb',
 'CNN_FATPAD_TransferLearning.ipynb',
 'Labels_FatPadDetection.xlsx',
 'Prepare_Xray_Data-CLAHE.ipynb',
 'Prepare_Xray_Data.ipynb',
 'README.md',
 'ResidualAttentionNetwork.py',
 'x-ray-images-enhancement',
 'xlearn',
 '__pycache__']

In [5]:
#change working directory to where the datasets are
root_directory=os.chdir('C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project')

In [6]:
#Provide train, validation, and test to tensorflow-keras ImageDataGenerator
#base_dir_train = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_clh_EFvsnEFSplit/train'
#base_dir_validate = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_clh_EFvsnEFSplit/validation'
#base_dir_test = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_clh_EFvsnEFSplit/test'


base_dir_train = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_ClaheSplit/train'
base_dir_validate = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_ClaheSplit/validation'
base_dir_test = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_ClaheSplit/test'

print(base_dir_train)
print(base_dir_train)

C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_ClaheSplit/train
C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_ClaheSplit/train


In [7]:
#run tensorflow-keras ImageDataGenerator on the directory which contains train, validate and test
#train_datagen = ImageDataGenerator(
 #   rescale=1./255, # new parameter (maps pixel values to floats)
 #   rotation_range=40,
 #   width_shift_range=0.2,
  ##  height_shift_range=0.2,
  #  shear_range=0.2,
  #  zoom_range=0.25,
  #  horizontal_flip=True,
  #  fill_mode='nearest') # Note: I added this, too

IMAGE_WIDTH=200
IMAGE_HEIGHT=200
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=1
IMAGE_SHAPE=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)
batch_size=16
epochs = 500
num_classes = 2

train_datagen = ImageDataGenerator(rescale=1./255,samplewise_center=True, 
              samplewise_std_normalization=True, 
              horizontal_flip = True, 
              vertical_flip = False, 
              height_shift_range= 0.05, 
              width_shift_range=0.05, 
              rotation_range=35, 
              shear_range = 0.41,
              fill_mode = 'nearest',
              zoom_range=0.25)


test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        base_dir_train,
       shuffle=True,
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    color_mode='rgb',
    batch_size=batch_size)

validation_generator = test_datagen.flow_from_directory(
        base_dir_validate, 
       target_size=IMAGE_SIZE,
    batch_size=batch_size,
    class_mode="categorical",
    color_mode='rgb',
    shuffle=True)

test_generator = train_datagen.flow_from_directory(
        base_dir_test,
        target_size=IMAGE_SIZE,
        batch_size=batch_size,
        class_mode="categorical",
        color_mode='rgb',
        shuffle=False)


Found 3227 images belonging to 2 classes.
Found 1211 images belonging to 2 classes.
Found 596 images belonging to 2 classes.


In [8]:
import matplotlib.pyplot as plt


def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()
    
def plot_hist_auc(hist):
    plt.plot(hist.history["auc"])
    plt.plot(hist.history["val_auc"])
    plt.title("model auc")
    plt.ylabel("auc")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

def summarize_diagnostics(history):
    # plot loss
    plt.subplot(411)
    plt.title('Cross Entropy Loss')
    plt.plot(history.history['loss'], color='blue', label='train')
    plt.plot(history.history['val_loss'], color='orange', label='validation')
    # plot accuracy
    plt.subplot(412)
    plt.title('Classification Accuracy')
    plt.plot(history.history['binary_accuracy'], color='blue', label='train')
    plt.plot(history.history['val_binary_accuracy'], color='orange', label='validation')
    
    plt.subplot(413)
    plt.title('Classification Precision')
    plt.plot(history.history['precision'], color='blue', label='train')
    plt.plot(history.history['val_precision'], color='orange', label='validation')
    
    plt.subplot(414)
    plt.title('Classification Recall')
    plt.plot(history.history['recall'], color='blue', label='train')
    plt.plot(history.history['val_recall'], color='orange', label='validation')
    # save plot to file
    plt.show()
    filename = sys.argv[0].split('/')[-1]
    plt.savefig(filename + '_plot.png')
    #plt.close()


In [9]:
train_generator.image_shape


(200, 200, 3)

In [10]:
initial_learning_rate = 0.0001
lr_schedule = tensorflow.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=False
)

In [11]:
checkpoint_cb = tensorflow.keras.callbacks.ModelCheckpoint("DenseNet_model_080321_acc.h5", save_best_only=True)

early_stopping_cb = tensorflow.keras.callbacks.EarlyStopping(
    patience=100, restore_best_weights=True
)

In [12]:
METRICS = [
    tensorflow.keras.metrics.BinaryAccuracy(),
    tensorflow.keras.metrics.Precision(name="precision"),
    tensorflow.keras.metrics.Recall(name="recall"),
]

In [15]:
from tensorflow.keras.applications import DenseNet121
inputs = layers.Input(shape=train_generator.image_shape)

#root_model = DenseNet121(include_top=True, input_tensor=inputs, weights="imagenet", classes=1000)
root_model = DenseNet121(include_top=True, input_tensor=inputs, weights=None, classes=1000)

root_model = tensorflow.keras.Model(inputs=root_model.input, outputs=root_model.layers[-5].output)

# Freeze the pretrained weights
root_model.trainable = False

# Rebuild top
model = models.Sequential()
model.add(root_model)
model.add(layers.BatchNormalization())
model.add(layers.GlobalAvgPool2D())
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation="softmax", name="pred"))

# Compile
#optimizer = tensorflow.keras.optimizers.Adam(learning_rate=lr_schedule)
optimizer = tensorflow.keras.optimizers.Adam(learning_rate=1e-4)

model.compile(loss='categorical_crossentropy', optimizer=optimizer,  metrics=METRICS)
#model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Functional)         (None, 6, 6, 1024)        7033408   
_________________________________________________________________
batch_normalization_1 (Batch (None, 6, 6, 1024)        4096      
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
pred (Dense)                 (None, 2)                 2050      
Total params: 7,039,554
Trainable params: 4,098
Non-trainable params: 7,035,456
________________________________________

In [16]:
batch_size=16
hist = model.fit(train_generator,
      epochs=100,
    steps_per_epoch=int( train_generator.n / batch_size),
      validation_data=validation_generator,
    validation_steps=int(validation_generator.n / batch_size),
      verbose=1, callbacks=[checkpoint_cb])

#plot_hist_auc(hist)


Epoch 1/100
182/201 [==========================>...] - ETA: 7s - loss: 0.6920 - binary_accuracy: 0.5405 - precision: 0.5405 - recall: 0.5405

KeyboardInterrupt: 

In [ ]:
summarize_diagnostics(hist)

In [ ]:
test_loss, test_acc, precision,recall = model.evaluate(test_generator, steps=int( test_generator.n / batch_size))


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from keras import metrics
Y_pred = model.predict(test_generator, verbose=1)
y_pred = np.argmax(Y_pred, axis=1)
print(y_pred)
class_labels = list(test_generator.class_indices.keys())   
print(class_labels)
true_classes = test_generator.classes

report = classification_report(true_classes, y_pred, target_names=class_labels)
print(report)

In [ ]:
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['Normal', 'Abnormal']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

In [ ]:
from sklearn.metrics import auc
from sklearn.metrics import roc_curve

fpr_keras, tpr_keras, thresholds_keras = roc_curve(test_generator.classes, Y_pred.argmax(axis=1))

auc_keras = auc(fpr_keras, tpr_keras)

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()
# Zoom in view of the upper left corner.
plt.figure(2)
plt.xlim(0, 0.4)
plt.ylim(0.6, 1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve (zoomed in at top left)')
plt.legend(loc='best')
plt.show()



In [ ]:
test_generator.classes
